In [179]:
from einops.einops import rearrange
import torch
mask = torch.randn((2,3,4))
mask_v, all_j_ids = mask.max(dim=2)  # mask_v: [N,L]
print(mask_v.shape,all_j_ids.shape)
b_ids, i_ids = torch.where(mask_v) #i_ids : [N*L]
print(b_ids.shape,i_ids.shape)
j_ids = all_j_ids[b_ids, i_ids]
print(j_ids)

i_ids = torch.arange(0,5).long()
print(i_ids)

torch.Size([2, 3]) torch.Size([2, 3])
torch.Size([6]) torch.Size([6])
tensor([1, 1, 1, 1, 2, 3])
tensor([0, 1, 2, 3, 4])


In [196]:
from einops.einops import rearrange
import torch
from kornia.utils import create_meshgrid
N =2
point2d = torch.Tensor([[[11.,3.3],[1,2.2],[5.6,3.8]],[[6.6,5.3],[5.5,9.8],[1.4,11.1]]])
print(point2d.shape)

point2d = point2d.round().long()
b_x = torch.flatten(point2d[:,:,0])
b_y = torch.flatten(point2d[:,:,1])

b,xx = torch.where(point2d[:,:,0]>=0)
print(xx.dtype)
#
device = point2d.device
grid = torch.zeros(2,12,12,device=device)

# print(mask)
#
#
print(b,b_y,b_x)
grid[b,b_y,b_x] =1
print(grid)
# torch.where(point2d)
# print(point2d)


# i_ids = torch.index_select(point2d, dim = 1, index = torch.tensor([0]))
#
# print(i_ids)
# print('point2d')
# print(point2d)
# print(point2d.shape)

# print(grid.shape)
# grid[point2d]  =1
# print(grid)


# x2 = torch.Tensor([[ 0.,  1.,  2.],
#                    [ 5.,  6.,  7.],
#                    [10., 11., 12.],
#                    [10., 11., 12.]])
#
# mask1 = torch.LongTensor([1,2])
# mask2 = torch.LongTensor([2,2])
# print(x2.shape)
# print(mask1.shape)
# x2[mask1,mask2] = -1
# print(x2)





torch.Size([2, 3, 2])
torch.int64
tensor([0, 0, 0, 1, 1, 1]) tensor([ 3,  2,  4,  5, 10, 11]) tensor([11,  1,  6,  7,  6,  1])
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [1]:

import os
import numpy as np
import torch
x = torch.ones((2,1,2,3))
heatmap_nms_batch = torch.cumsum(x,2)
heatmap_nms_batch[::3] =0
print(heatmap_nms_batch)
pts_idx = heatmap_nms_batch[...].nonzero()  # [N, 4(batch, 0, y, x)]
# print(pts_idx)
y = torch.topk(heatmap_nms_batch,1,largest=True,sorted=True)
print(y)

tensor([[[[0., 0., 0.],
          [0., 0., 0.]]],


        [[[1., 1., 1.],
          [2., 2., 2.]]]])
torch.return_types.topk(
values=tensor([[[[0.],
          [0.]]],


        [[[1.],
          [2.]]]]),
indices=tensor([[[[0],
          [0]]],


        [[[0],
          [0]]]]))


In [63]:
import math
import torch
from torch import nn


class PositionEncodingSine(nn.Module):
    """
    This is a sinusoidal position encoding that generalized to 2-dimensional images
    """

    def __init__(self, d_model, max_shape=(512, 512), temp_bug_fix=True):
        """
        Args:
            max_shape (tuple): for 1/8 featmap, the max length of 256 corresponds to 2048 pixels
            temp_bug_fix (bool): As noted in this [issue](https://github.com/zju3dv/LoFTR/issues/41),
                the original implementation of LoFTR includes a bug in the pos-enc impl, which has little impact
                on the final performance. For now, we keep both impls for backward compatability.
                We will remove the buggy impl after re-training all variants of our released models.
        """
        super().__init__()

        pe = torch.zeros((d_model, *max_shape))
        y_position = torch.ones(max_shape).cumsum(0).float().unsqueeze(0)
        x_position = torch.ones(max_shape).cumsum(1).float().unsqueeze(0)
        if temp_bug_fix:
            div_term = torch.exp(torch.arange(0, d_model//2, 2).float() * (-math.log(10000.0) / (d_model//2)))
        else:  # a buggy implementation (for backward compatability only)
            div_term = torch.exp(torch.arange(0, d_model//2, 2).float() * (-math.log(10000.0) / d_model//2))
        div_term = div_term[:, None, None]  # [C//4, 1, 1]
        pe[0::4, :, :] = torch.sin(x_position * div_term)
        pe[1::4, :, :] = torch.cos(x_position * div_term)
        pe[2::4, :, :] = torch.sin(y_position * div_term)
        pe[3::4, :, :] = torch.cos(y_position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0), persistent=False)  # [1, C, H, W]

    def forward(self, x):
        """
        Args:
            x: [N, C, H, W]
        """
        return x + self.pe[:, :, :x.size(2), :x.size(3)]

class PositionEncodingSine_line(nn.Module):
    """
    This is a sinusoidal position encoding that generalized to 1-dimensional sequences
    """

    def __init__(self, d_model, temp_bug_fix=True):
        """
        Args:
            max_shape (tuple): for 1/8 featmap, the max length of 256 corresponds to 2048 pixels
            temp_bug_fix (bool): As noted in this [issue](https://github.com/zju3dv/LoFTR/issues/41),
                the original implementation of LoFTR includes a bug in the pos-enc impl, which has little impact
                on the final performance. For now, we keep both impls for backward compatability.
                We will remove the buggy impl after re-training all variants of our released models.
        """
        super().__init__()
        if temp_bug_fix:
            div_term = torch.exp(torch.arange(0, d_model//2, 2).float() * (-math.log(10000.0) / (d_model//2)))
        else:  # a buggy implementation (for backward compatability only)
            div_term = torch.exp(torch.arange(0, d_model//2, 2).float() * (-math.log(10000.0) / d_model//2))

        self.div_term = div_term[:,None, None]  # [C//4, 1]





    def forward(self, x, pts_int):
        """
        Args:
            x: [bs, C, L]
            pts_int:[bs,L,2]
        """
        d_model = x.shape[1]
        x_position = pts_int[:,:,0].unsqueeze(0)
        y_position = pts_int[:,:,0].unsqueeze(0)

        pe = torch.zeros((x.shape[0],d_model, x.shape[2]))
        print(pe.size())
        b = torch.sin(x_position * self.div_term).permute((1,0,2))
        print(b.size())
        pe[:,0::4, :] = torch.sin(x_position * self.div_term).permute((1,0,2))
        pe[:,1::4, :] = torch.cos(x_position * self.div_term).permute((1,0,2))
        pe[:,2::4, :] = torch.sin(y_position * self.div_term).permute((1,0,2))
        pe[:,3::4, :] = torch.cos(y_position * self.div_term).permute((1,0,2))
        return x + pe


model = PositionEncodingSine(16)
x = torch.zeros(1,16,2,2)
y = model(x)
print('y size',y.size())
# print(y)
model2 = PositionEncodingSine_line(16)
x = torch.zeros(2,16,4)

pts_int = torch.tensor([[[  0.,   0.],
         [  0.,   1.],
         [  1.,   0.],[  1.,   1.]],
               [[  1.,   1.],
         [  1.,   0.],
         [  1.,   1.],[  1.,   2.]]
                        ])
pts_int = pts_int +1

y = model2(x,pts_int)
print('y size',y.size())
print(y)





y size torch.Size([1, 16, 2, 2])
torch.Size([2, 16, 4])
torch.Size([2, 4, 4])
y size torch.Size([2, 16, 4])
tensor([[[ 0.8415,  0.8415,  0.9093,  0.9093],
         [ 0.5403,  0.5403, -0.4161, -0.4161],
         [ 0.8415,  0.8415,  0.9093,  0.9093],
         [ 0.5403,  0.5403, -0.4161, -0.4161],
         [ 0.0998,  0.0998,  0.1987,  0.1987],
         [ 0.9950,  0.9950,  0.9801,  0.9801],
         [ 0.0998,  0.0998,  0.1987,  0.1987],
         [ 0.9950,  0.9950,  0.9801,  0.9801],
         [ 0.0100,  0.0100,  0.0200,  0.0200],
         [ 0.9999,  0.9999,  0.9998,  0.9998],
         [ 0.0100,  0.0100,  0.0200,  0.0200],
         [ 0.9999,  0.9999,  0.9998,  0.9998],
         [ 0.0010,  0.0010,  0.0020,  0.0020],
         [ 1.0000,  1.0000,  1.0000,  1.0000],
         [ 0.0010,  0.0010,  0.0020,  0.0020],
         [ 1.0000,  1.0000,  1.0000,  1.0000]],

        [[ 0.9093,  0.9093,  0.9093,  0.9093],
         [-0.4161, -0.4161, -0.4161, -0.4161],
         [ 0.9093,  0.9093,  0.9093,  0.9093

In [198]:
import torch
a = torch.ones(1,2,3)
print(a)
a.sum(-1)

tensor([[[1., 1., 1.],
         [1., 1., 1.]]])


tensor([[3., 3.]])

In [40]:
import torch
a = torch.rand(1,4,2)
# print(a)
bias = torch.randn(1,2)
print(bias[:,0,None])
# print(bias[:,0])
# print( a[:,:,0].shape)
# print( bias[:,0].shape)
scale_x = torch.randn(1)
print(scale_x[:,None])
bias[:,0,None]*scale_x[:,None]
# x = a[:,:,0] +bias[:,0,None]/scale_x[:,None]
# y = a[:,:,1] ++bias[:,1,None]/scale_x[:,None]
# z = torch.stack((x,y),dim=-1)
# print(z)

tensor([[0.4814]])
tensor([[-0.3652]])


tensor([[-0.1758]])